In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_parquet("s3://dev-ds-clean-data-1654/clean_data.parquet")
df.head()

,attrition_flag,customer_age,gender,dependent_count,education_level,marital_status,income_category,card_category,months_on_book,total_relationship_count,months_inactive_12_mon,contacts_count_12_mon,credit_limit,total_revolving_bal,avg_open_to_buy,total_amt_chng_q4_q1,total_trans_amt,total_trans_ct,total_ct_chng_q4_q1,avg_utilization_ratio
0,existing customer,45,m,3,high school,married,$60k - $80k,blue,39,5,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,existing customer,49,f,5,graduate,single,less than $40k,blue,44,6,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,existing customer,51,m,3,graduate,married,$80k - $120k,blue,36,4,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,existing customer,40,f,4,high school,unknown,less than $40k,blue,34,3,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,existing customer,40,m,3,uneducated,married,$60k - $80k,blue,21,5,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   attrition_flag            10127 non-null  object 
 1   customer_age              10127 non-null  int64  
 2   gender                    10127 non-null  object 
 3   dependent_count           10127 non-null  int64  
 4   education_level           10127 non-null  object 
 5   marital_status            10127 non-null  object 
 6   income_category           10127 non-null  object 
 7   card_category             10127 non-null  object 
 8   months_on_book            10127 non-null  int64  
 9   total_relationship_count  10127 non-null  int64  
 10  months_inactive_12_mon    10127 non-null  int64  
 11  contacts_count_12_mon     10127 non-null  int64  
 12  credit_limit              10127 non-null  float64
 13  total_revolving_bal       10127 non-null  int64  
 14  avg_op

In [4]:
df['attrition_flag'].unique()

array(['existing customer', 'attrited customer'], dtype=object)

In [5]:
df['attrition_flag'] = df['attrition_flag'].map({
    'existing customer': 0,
    'attrited customer': 1
})

In [6]:
from sklearn.preprocessing import OneHotEncoder

gender_ohe = OneHotEncoder(drop='first', sparse_output=False)
gender_encoded = gender_ohe.fit_transform(df[['gender']])
gender_cols = gender_ohe.get_feature_names_out(['gender'])
df_gender = pd.DataFrame(gender_encoded, columns=gender_cols, index=df.index)
df = pd.concat([df.drop(columns=['gender']), df_gender], axis=1)

In [7]:
from sklearn.preprocessing import OrdinalEncoder

education_order = [
    'unknown',
    'uneducated',
    'high school',
    'college',
    'graduate',
    'post-graduate',
    'doctorate'
]

edu_encoder = OrdinalEncoder(
    categories=[education_order],
    handle_unknown='use_encoded_value',
    unknown_value=-1
)

df['education_level'] = edu_encoder.fit_transform(
    df[['education_level']]
)

In [8]:
marital_ohe = OneHotEncoder(drop='first', sparse_output=False)
marital_encoded = marital_ohe.fit_transform(df[['marital_status']])
marital_cols = marital_ohe.get_feature_names_out(['marital_status'])
df_marital = pd.DataFrame(marital_encoded, columns=marital_cols, index=df.index)
df = pd.concat([df.drop(columns=['marital_status']), df_marital], axis=1)

In [9]:
income_order = [
    "unknown",
    "less than $40k",
    "$40k - $60k",
    "$60k - $80k",
    "$80k - $120k",
    "$120k +"
]

income_encoder = OrdinalEncoder(
    categories=[income_order],
    handle_unknown='use_encoded_value',
    unknown_value=-1
)

df['income_category'] = income_encoder.fit_transform(
    df[['income_category']]
)

In [10]:
card_order = ["blue", "silver", "gold", "platinum"]

card_encoder = OrdinalEncoder(
    categories=[card_order],
    handle_unknown='use_encoded_value',
    unknown_value=-1
)

df['card_category'] = card_encoder.fit_transform(
    df[['card_category']]
)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   attrition_flag            10127 non-null  int64  
 1   customer_age              10127 non-null  int64  
 2   dependent_count           10127 non-null  int64  
 3   education_level           10127 non-null  float64
 4   income_category           10127 non-null  float64
 5   card_category             10127 non-null  float64
 6   months_on_book            10127 non-null  int64  
 7   total_relationship_count  10127 non-null  int64  
 8   months_inactive_12_mon    10127 non-null  int64  
 9   contacts_count_12_mon     10127 non-null  int64  
 10  credit_limit              10127 non-null  float64
 11  total_revolving_bal       10127 non-null  int64  
 12  avg_open_to_buy           10127 non-null  float64
 13  total_amt_chng_q4_q1      10127 non-null  float64
 14  total_

In [12]:
X = df.drop(columns=['attrition_flag'])
y = df['attrition_flag']

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [14]:
!pip install "xgboost<2.1"

In [15]:
from xgboost import XGBClassifier

In [19]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import numpy as np

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

auc_scores = []

for fold, (train_idx, val_idx) in enumerate(cv.split(X_train, y_train)):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

    model = XGBClassifier(
        n_estimators=300,
        learning_rate=0.1,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="binary:logistic",
        eval_metric="auc",
        scale_pos_weight=(y_tr == 0).sum() / (y_tr == 1).sum(),
        random_state=42,
        n_jobs=-1
    )

    model.fit(X_tr, y_tr)

    y_val_proba = model.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, y_val_proba)
    auc_scores.append(auc)

    print(f"Fold {fold+1} AUC: {auc:.4f}")

print("CV AUC scores:", auc_scores)
print("Mean CV AUC:", np.mean(auc_scores))


Fold 1 AUC: 0.9949
Fold 2 AUC: 0.9940
Fold 3 AUC: 0.9930
Fold 4 AUC: 0.9952
Fold 5 AUC: 0.9907
CV AUC scores: [0.9948980166779355, 0.9940356334841629, 0.992975113122172, 0.9952403846153847, 0.9907047947696497]
Mean CV AUC: 0.9935707885338608


In [20]:
xgb.fit(X_train, y_train)

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.8
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'auc'


In [21]:
from sklearn.metrics import roc_auc_score, classification_report

y_prob = xgb.predict_proba(X_test)[:, 1]
y_pred = (y_prob >= 0.5).astype(int)

print("Test ROC-AUC:", roc_auc_score(y_test, y_prob))
print(classification_report(y_test, y_pred))

Test ROC-AUC: 0.9931569664902998
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      1701
           1       0.93      0.90      0.91       325

    accuracy                           0.97      2026
   macro avg       0.95      0.94      0.95      2026
weighted avg       0.97      0.97      0.97      2026

